In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
import os
import pickle

***ПУТЬ ДО ФАЙЛА.. НУЖНО ПОМЕНЯТЬ***

In [2]:
DATA_PATH = "/media/andrey/big/datasets/XMAS/X_train.csv"

In [3]:
data = pd.read_csv(DATA_PATH)
data

,eventTime,email,ip,fingerprint,cardToken,paymentSystem,providerId,bankCountry,partyId,shopId,amount,currency,result,cnt,sum,bin_hash,ms_pan_hash
0,2020-02-11 06:36:23,25ffcc80116825b7166effa3861ff923ac10907ee0d8f6...,a269dd68eaad160623142377811baaf8e02405a2026e0a...,NaN,8dcc0a7fc2a190773acea84b5bde2715080f283fdb28e1...,visa,6affdae3b3c1aa6aa7689e9b6a7b3225a636aa1ac0025f...,RUS,9e8d0b37f497e0abfb4735532f506da9ac23813f46cbb0...,5d5298941c184ce67c6ec52d472bae29d33f533344d259...,10000,RUB,False,10000.0,1.0,771e23c8c0fe2e38412ab9b8306bf6f27212dacb483bd0...,7e73164c9aee2f54ee16785dbd7995d22a9dbf1912623c...
1,2020-02-11 04:23:53,NaN,NaN,NaN,145417ba9e7aeaed3de8eb2f452cb0b4708449f54f2093...,mastercard,9bdb2af6799204a299c603994b8e400e4b1fd625efdb74...,RUS,9e8d0b37f497e0abfb4735532f506da9ac23813f46cbb0...,52daceda3a64caf0a19a69f5138fe6c514b0afec9966e8...,35000,RUB,False,1020000.0,22.0,00e8a97abcbb2b4ff83a26a8eb4c2be9b3306e2ec22fb8...,04599177e5e8f329d50fb934fd52c34aa6562e284f7685...
2,2020-02-11 08:04:02,94b6b23c41fd0f50cc066c66d9b21d932483c89c1de9a6...,8d571220e913f729cb3aa8959e5e0dd7bb792885ab7d7f...,NaN,32cabea0847650745bb67d4409a220c83541d9b466bf88...,nspkmir,ad57366865126e55649ecb23ae1d48887544976efea46a...,RUS,b8c69eead38227c90ca20016a908ec9ebc096e36da0f5b...,b461dbe80f1c66a102687630af75a181ab7060e5330fce...,90000,RUB,False,90000.0,1.0,bff0bbfd523067a0cecaa93f130626db4dc92e1580ca7c...,be11f064361c33e703bd1f3562b106cb28f6829fb4b3ad...
3,2020-02-11 12:15:02,NaN,3b9a3ef525c6b09cb5995de94e4b8b2f75b059bd9e911f...,NaN,d8d09c692874e8b7a9d6354d53b9b80a599bbfae95234b...,mastercard,9bdb2af6799204a299c603994b8e400e4b1fd625efdb74...,RUS,339cb819cd996ec22ad75467898a2b851a448c11059515...,034de6b2c7f60bc3146b9da67bf56a4aa14a0879a7330d...,30000,RUB,False,200000.0,6.0,d14a028c2a3a2bc9476102bb288234c415a2b01f828ea6...,38a978ce0bfcd461f79966342e90bb5ffb29868d2a3a6e...
4,2020-02-11 19:27:31,NaN,3b9a3ef525c6b09cb5995de94e4b8b2f75b059bd9e911f...,NaN,0f4667dd01c5a114685bd74fb486cb9131b0cb236de587...,visa,9f1f9dce319c4700ef28ec8c53bd3cc8e6abe64c683854...,RUS,339cb819cd996ec22ad75467898a2b851a448c11059515...,034de6b2c7f60bc3146b9da67bf56a4aa14a0879a7330d...,400000,RUB,False,420000.0,2.0,e9e9ab8de4cc6e1c81247fdb1ce0a037de921ccdedc5cd...,33977f6186f639e90bd6ee97ad6f1918a93b72d8b5b6a2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,2020-02-11 17:46:52,NaN,NaN,NaN,90caa2a4ff2c056dbb99acb8de9f0c2d3d1359ae25a95a...,visa,6affdae3b3c1aa6aa7689e9b6a7b3225a636aa1ac0025f...,RUS,9e8d0b37f497e0abfb4735532f506da9ac23813f46cbb0...,5d5298941c184ce67c6ec52d472bae29d33f533344d259...,50000,RUB,False,1370000.0,13.0,fc98140d09b81335c8c61bcdc16eaacd166342ddf913c7...,45929da8e649d91e495f382cc6539ce7725c5633030fd3...
159996,2020-02-11 15:37:23,10dabab5a3d7b63c6211928df29ac120bffe531d3c09c5...,c55cf9d07a38056638a1b1aff095ac328fe03ae02c3de9...,NaN,55e0ef4c95cc0523e2241b6a756d9dfcb57253038277bc...,visa,9bdb2af6799204a299c603994b8e400e4b1fd625efdb74...,RUS,186f308113ac7e6a67cc0145303bd31663f44f8981e10c...,d3a78976a03b14e5f54b89a8d0ec4cc0d4777994b4a0fd...,10000,RUB,False,10000.0,1.0,5aee0c5e4c1bb59d3eddbdbfacddd58bd64af3fe8ce145...,78c39702e6e430849bcbfe34c008d8865b468f100a6d00...
159997,2020-02-11 19:14:39,222e02b017875d5d0dc12f6b28965d8a5d62667f0b395b...,46c830a5364583e82adf5494ebe14146ffbbcd3b2a99df...,NaN,f53ed6e0fec6b6d687e1f38930d6f02ac203fc3496d93c...,nspkmir,9bdb2af6799204a299c603994b8e400e4b1fd625efdb74...,RUS,313a15ed377795c5fb06c80506f361c855a0a7b7d9c27d...,a8cea4ad4d0df7526e5a21dd9678f39aa8ac505a27b1f6...,50000,RUB,False,50000.0,1.0,98c2555994c5e42a686b5d0c5b16b8769b23751d26d39a...,4d187d1cd63df7fe7831eeb94acf13bd338e354b5d789c...
159998,2020-02-11 17:11:51,NaN,6088326a27d6f4d4d446c1a66cfde81f307fa3685de746...,NaN,a1bc8d9235e8e829501f7ff86883c54d316d012ff82f04...,visa,9bdb2af6799204a299c603994b8e400e4b1fd625efdb74...,RUS,313a15ed377795c5fb06c80506f361c855a0a7b7d9c27d...,cb17696a481570a045977e96b6d8d6344470078e4fbb43...,5000,RUB,False,500

In [4]:
for col in data.columns:
    print(col, "UNIQUE COUNT:", len(data[col].unique()), "NULL:", data[col].isnull().values.any(), "NULL_PROB:", data[col].isnull().sum() / len(data))

eventTime UNIQUE COUNT: 62874 NULL: False NULL_PROB: 0.0
email UNIQUE COUNT: 23297 NULL: True NULL_PROB: 0.69229375
ip UNIQUE COUNT: 13244 NULL: True NULL_PROB: 0.2360375
fingerprint UNIQUE COUNT: 1 NULL: True NULL_PROB: 1.0
cardToken UNIQUE COUNT: 105527 NULL: False NULL_PROB: 0.0
paymentSystem UNIQUE COUNT: 5 NULL: False NULL_PROB: 0.0
providerId UNIQUE COUNT: 14 NULL: False NULL_PROB: 0.0
bankCountry UNIQUE COUNT: 114 NULL: True NULL_PROB: 0.0003125
partyId UNIQUE COUNT: 207 NULL: False NULL_PROB: 0.0
shopId UNIQUE COUNT: 415 NULL: False NULL_PROB: 0.0
amount UNIQUE COUNT: 5519 NULL: False NULL_PROB: 0.0
currency UNIQUE COUNT: 3 NULL: False NULL_PROB: 0.0
result UNIQUE COUNT: 2 NULL: False NULL_PROB: 0.0
cnt UNIQUE COUNT: 13148 NULL: True NULL_PROB: 0.0099125
sum UNIQUE COUNT: 173 NULL: True NULL_PROB: 0.0099125
bin_hash UNIQUE COUNT: 2961 NULL: False NULL_PROB: 0.0
ms_pan_hash UNIQUE COUNT: 10032 NULL: False NULL_PROB: 0.0


***ДЛЯ ТОГО ЧТОБЫ ПОСТРОИЛАСЬ СТАТИСТИКА НА СКРЫТЫХ ДАННЫХ РАСКОМЕНТИТЕ СТРОЧКИ (СПЕЦИАЛЬНО ЗАКОМЕНТИЛ ДЛЯ ФИНАЛЬНОЙ МОДЕЛИ ПО ВСЕМ СТРОЧКАМ)***

In [5]:
train_data = data
test_data = data
# msk = np.random.rand(len(data)) < 0.9
# train_data = data[msk]
# test_data = data[~msk]

In [6]:
def calculate_stats(df):
    return {
        "email": df[df.email.notnull()].groupby("email").mean(),
        "ip": df[df.ip.notnull()].groupby("ip").mean(),
        "cardToken": df[df.cardToken.notnull()].groupby("cardToken").mean(),
        "paymentSystem": df[df.cardToken.notnull()].groupby("cardToken").mean(),
        "providerId": df[df.providerId.notnull()].groupby("providerId").mean(),
        "bankCountry": df[df.bankCountry.notnull()].groupby("bankCountry").mean(),
        "partyId": df[df.partyId.notnull()].groupby("partyId").mean(),
        "shopId": df[df.shopId.notnull()].groupby("shopId").mean(),
        "currency": df[df.currency.notnull()].groupby("currency").mean(),
        "bin_hash": df[df.bin_hash.notnull()].groupby("bin_hash").mean(),
        "ms_pan_hash": df[df.ms_pan_hash.notnull()].groupby("ms_pan_hash").mean()
    }
    

In [7]:
def get_cat_feature(df, col_name, stat):
    return df[col_name].map(lambda x: stat[col_name].loc[x].result if x in stat[col_name].index else stat[col_name].result.mean()).values
def get_null_feature(df, col_name, stat):
    return df[col_name].map(lambda x: x in stat[col_name].index).values.astype(np.float32)

In [8]:
def generate_features(df, stat):
    features = []
    for col_name in ["email","ip","cardToken","paymentSystem",
                     "providerId","bankCountry","partyId","shopId",
                     "currency","bin_hash","ms_pan_hash"]:
        features.append(get_cat_feature(df, col_name, stat))
        #features.append(get_null_feature(df, col_name, stat))
    
    #features.append(df.amount.fillna(-1).values)
    features.append(df.amount * df.currency.map({"RUB":1, "USD": 63, "EUR": 63}))
    #features.append(df.cnt.fillna(-1).values)
    #features.append(df["sum"].fillna(-1).values)
    
    return np.array(features).T

In [9]:
def generate_new_data_with_nulls(df):
    new_df = []
    for i in range(2):
        copy_df = df.copy()
        copy_df.loc[np.random.rand(len(copy_df)) < 0.5, "email"] = np.nan
        copy_df.loc[np.random.rand(len(copy_df)) < 0.2, "ip"] = np.nan
        copy_df.loc[np.random.rand(len(copy_df)) < 0.02, "bankCountry"] = np.nan
        new_df.append(copy_df)
    return pd.concat(new_df)

In [10]:
for FOLD in range(20):
    print("PROCESS FOLD:", FOLD)
    print("SPLIT DATA...")
    rnd_arr = np.random.rand(len(train_data))
    msk_stat = rnd_arr < 0.5
    msk_train = ((rnd_arr >= 0.5) & (rnd_arr < 0.9))
    msk_test = rnd_arr >= 0.9
    stat_dataframe = train_data[msk_stat]
    train_dataframe = generate_new_data_with_nulls(train_data[msk_train])
    test_dataframe = train_data[msk_test]
    
    print("PREPROCESS DATA..")
    stat = calculate_stats(stat_dataframe)
    features = generate_features(train_dataframe, stat)
    target = train_dataframe.result.values.astype(np.float32)

    features_test = generate_features(test_dataframe, calculate_stats(train_dataframe))
    target_test = test_dataframe.result.values.astype(np.float32)
    
    print("FRAUD PROBABILITY:", target_test.sum() / len(target_test))
    
    print("Training...")
    params = {'loss_function': "Logloss", 
          'verbose': 100, 
          'random_seed': 1,
          "iterations":5000,
          'od_type' : "Iter",
          'od_wait' : 800,
          'eval_metric': 'PRAUC',
          'depth': 3,
         }

    pool_train = Pool(features, target)
    pool_test = Pool(features_test, target_test)

    cbc = CatBoostClassifier(**params)
    cbc.fit(pool_train, eval_set=pool_test);
    cbc.save_model("models/" + str(FOLD) + "_model.cbt")

PROCESS FOLD: 0
SPLIT DATA...
PREPROCESS DATA..
FRAUD PROBABILITY: 0.013870827915041179
Training...
Learning rate set to 0.052186
0:	learn: 0.9633188	test: 0.9706579	best: 0.9706579 (0)	total: 61ms	remaining: 5m 4s
100:	learn: 0.9992714	test: 0.9985344	best: 0.9985623 (91)	total: 1.27s	remaining: 1m 1s
200:	learn: 0.9997870	test: 0.9974381	best: 0.9986803 (111)	total: 2.58s	remaining: 1m 1s
300:	learn: 0.9998422	test: 0.9970794	best: 0.9986803 (111)	total: 3.91s	remaining: 1m 1s
400:	learn: 0.9998435	test: 0.9970410	best: 0.9986803 (111)	total: 5.18s	remaining: 59.4s
500:	learn: 0.9998435	test: 0.9970410	best: 0.9986803 (111)	total: 6.41s	remaining: 57.5s
600:	learn: 0.9998435	test: 0.9970410	best: 0.9986803 (111)	total: 7.63s	remaining: 55.9s
700:	learn: 0.9998435	test: 0.9970410	best: 0.9986803 (111)	total: 8.86s	remaining: 54.3s
800:	learn: 0.9998435	test: 0.9970410	best: 0.9986803 (111)	total: 10.1s	remaining: 52.7s
900:	learn: 0.9998435	test: 0.9970410	best: 0.9986803 (111)	total:

500:	learn: 0.9989575	test: 0.9926973	best: 0.9930428 (288)	total: 6.28s	remaining: 56.4s
600:	learn: 0.9989575	test: 0.9926973	best: 0.9930428 (288)	total: 9.53s	remaining: 1m 9s
700:	learn: 0.9989575	test: 0.9926973	best: 0.9930428 (288)	total: 14.6s	remaining: 1m 29s
800:	learn: 0.9989575	test: 0.9926973	best: 0.9930428 (288)	total: 16.6s	remaining: 1m 26s
900:	learn: 0.9989575	test: 0.9926973	best: 0.9930428 (288)	total: 17.9s	remaining: 1m 21s
1000:	learn: 0.9989575	test: 0.9926973	best: 0.9930428 (288)	total: 19.3s	remaining: 1m 17s
Stopped by overfitting detector  (800 iterations wait)

bestTest = 0.9930428245
bestIteration = 288

Shrink model to first 289 iterations.
PROCESS FOLD: 7
SPLIT DATA...
PREPROCESS DATA..
FRAUD PROBABILITY: 0.013174404015056462
Training...
Learning rate set to 0.05219
0:	learn: 0.9724998	test: 0.9669042	best: 0.9669042 (0)	total: 12ms	remaining: 1m
100:	learn: 0.9968444	test: 0.9924129	best: 0.9924836 (99)	total: 1.02s	remaining: 49.5s
200:	learn: 0.99

400:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 4.8s	remaining: 55.1s
500:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 5.93s	remaining: 53.3s
600:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 7.07s	remaining: 51.7s
700:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 8.2s	remaining: 50.3s
800:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 9.33s	remaining: 48.9s
900:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 10.5s	remaining: 47.6s
1000:	learn: 0.9999472	test: 0.9946384	best: 0.9946493 (297)	total: 11.6s	remaining: 46.3s
Stopped by overfitting detector  (800 iterations wait)

bestTest = 0.9946492516
bestIteration = 297

Shrink model to first 298 iterations.
PROCESS FOLD: 13
SPLIT DATA...
PREPROCESS DATA..
FRAUD PROBABILITY: 0.01310396224071544
Training...
Learning rate set to 0.052104
0:	learn: 0.9704802	test: 0.9632751	best: 0.9632751 (0)	total: 10ms	remaining: 50.2s
100:	learn: 0.998

100:	learn: 0.9990622	test: 0.9908322	best: 0.9913865 (73)	total: 1.1s	remaining: 53.4s
200:	learn: 0.9997392	test: 0.9903847	best: 0.9913865 (73)	total: 2.42s	remaining: 57.8s
300:	learn: 0.9999042	test: 0.9902983	best: 0.9913865 (73)	total: 3.78s	remaining: 59s
400:	learn: 0.9999111	test: 0.9902873	best: 0.9913865 (73)	total: 5.08s	remaining: 58.3s
500:	learn: 0.9999111	test: 0.9902873	best: 0.9913865 (73)	total: 6.32s	remaining: 56.7s
600:	learn: 0.9999111	test: 0.9902873	best: 0.9913865 (73)	total: 7.55s	remaining: 55.3s
700:	learn: 0.9999111	test: 0.9902873	best: 0.9913865 (73)	total: 8.78s	remaining: 53.9s
800:	learn: 0.9999111	test: 0.9902873	best: 0.9913865 (73)	total: 10s	remaining: 52.5s
Stopped by overfitting detector  (800 iterations wait)

bestTest = 0.9913864826
bestIteration = 73

Shrink model to first 74 iterations.


In [11]:
global_stat = calculate_stats(data)
for_test_stat = calculate_stats(train_data)

In [12]:
with open(os.path.join("models", 'global_stat.pickle'), 'wb') as handle:
    pickle.dump(global_stat, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
test_features = generate_features(test_data, for_test_stat)
test_targets = test_data.result.values.astype(np.float32)

In [16]:
results = []
for model_name in os.listdir("models"):
    print("PREDICT MODEL:", model_name)
    if "model.cbt" not in model_name: continue
    cbc = CatBoostClassifier()
    cbc.load_model(os.path.join("models", model_name))
    res = cbc.predict_proba(test_features)[:, 1]
    results.append(res)
results = (np.array(results).mean(axis=0) > 0.2)

PREDICT MODEL: 12_model.cbt
PREDICT MODEL: 5_model.cbt
PREDICT MODEL: 15_model.cbt
PREDICT MODEL: 1_model.cbt
PREDICT MODEL: 2_model.cbt
PREDICT MODEL: 10_model.cbt
PREDICT MODEL: 17_model.cbt
PREDICT MODEL: 19_model.cbt
PREDICT MODEL: 11_model.cbt
PREDICT MODEL: 7_model.cbt
PREDICT MODEL: 16_model.cbt
PREDICT MODEL: 18_model.cbt
PREDICT MODEL: 8_model.cbt
PREDICT MODEL: 0_model.cbt
PREDICT MODEL: 9_model.cbt
PREDICT MODEL: 13_model.cbt
PREDICT MODEL: 14_model.cbt
PREDICT MODEL: 6_model.cbt
PREDICT MODEL: global_stat.pickle
PREDICT MODEL: 3_model.cbt
PREDICT MODEL: 4_model.cbt


In [17]:

amount_currency = test_data.amount * test_data.currency.map({"RUB":1, "USD": 63, "EUR": 63})
print("TEST DATASET STATISTICS...")
print("Количество транзакций:", len(test_data))
print("Количество фрод транзакций:", test_data.result.sum())
print("Процент мошенничиских сделок:", 100 * test_data.result.sum() / len(test_data), "%")
print("Транзакций на сумму (с учетом стоимости EUR и USD по 63р):", amount_currency.sum(), "руб. ")
print("Потери от фрода (с учетом стоимости EUR и USD по 63р):", amount_currency[test_data.result].sum(), "руб. ")
print("Прибыль компании (с учетом стоимости EUR и USD по 63р):", amount_currency[~test_data.result].sum() - amount_currency[test_data.result].sum(), "руб. ")
print("-"*100)

filtered_fraud_data = test_data[~results]
droped_fraud_data = test_data[results]
amount_currency = filtered_fraud_data.amount * filtered_fraud_data.currency.map({"RUB":1, "USD": 63, "EUR": 63})
print("После применения Anti-fraud системы...")
print("Количество транзакций:", len(filtered_fraud_data))
print("Количество фрод транзакций:", filtered_fraud_data.result.sum())
print("Процент мошенничиских сделок:", 100 * filtered_fraud_data.result.sum() / len(test_data), "%")

print("Транзакций на сумму (с учетом стоимости EUR и USD по 63р):", amount_currency.sum(), "руб. ")
print("Потери от фрода (с учетом стоимости EUR и USD по 63р):", amount_currency[filtered_fraud_data.result].sum(), "руб. ")
print("Прибыль компании (с учетом стоимости EUR и USD по 63р):", amount_currency[~filtered_fraud_data.result].sum() - amount_currency[filtered_fraud_data.result].sum(), "руб. ")
print("-"*100)
print("Количество обнаруженных фрод транзакций:", droped_fraud_data.result.sum())
print("Количество транзакций, обнаруженных как фрод, но не являющиеся такими:", len(droped_fraud_data) - droped_fraud_data.result.sum())

TEST DATASET STATISTICS...
Количество транзакций: 160000
Количество фрод транзакций: 2228
Процент мошенничиских сделок: 1.3925 %
Транзакций на сумму (с учетом стоимости EUR и USD по 63р): 21403835612 руб. 
Потери от фрода (с учетом стоимости EUR и USD по 63р): 1036556715 руб. 
Прибыль компании (с учетом стоимости EUR и USD по 63р): 19330722182 руб. 
----------------------------------------------------------------------------------------------------
После применения Anti-fraud системы...
Количество транзакций: 157774
Количество фрод транзакций: 3
Процент мошенничиских сделок: 0.001875 %
Транзакций на сумму (с учетом стоимости EUR и USD по 63р): 20383113881 руб. 
Потери от фрода (с учетом стоимости EUR и USD по 63р): 15900000 руб. 
Прибыль компании (с учетом стоимости EUR и USD по 63р): 20351313881 руб. 
----------------------------------------------------------------------------------------------------
Количество обнаруженных фрод транзакций: 2225
Количество транзакций, обнаруженных как